In [ ]:
#| default_exp widget

In [ ]:
#| export
from __future__ import annotations

# Widget
> FashHTML widgets


# Prologue

In [ ]:
#| export
import dataclasses
import typing
from contextlib import contextmanager
from dataclasses import dataclass
from dataclasses import field
from dataclasses import KW_ONLY
from functools import cached_property
from functools import partial
from types import MethodType
from typing import Any
from typing import Callable
from typing import ClassVar
from typing import Iterable
from typing import Sequence
from typing import Type

import psygnal
import traitlets as T
from fastcore.xml import FT
from fastcore.xml import to_xml
from fasthtml.core import FastHTML
from olio.common import val_at
from psygnal import Signal


In [ ]:
#| export
from bridget.bridge_plugins import NBHooksPlugin
from bridget.bridget import Bridgeable
from bridget.common import *
from bridget.helpers import id_gen
from bridget.nb_state import NBStateFeedback
from bridget.routing import APIRouterD


In [ ]:
#| export

import fasthtml.components as H
from fasthtml.components import Div, Label, Input, Text, I, Select, Option, Span, show


In [ ]:
import fastcore.all as FC
import ipywidgets as W
from fastcore.test import *
from fastcore.xml import highlight
from fastcore.xml import showtags
from fasthtml.components import Button
from IPython.display import DisplayHandle
from IPython.core.display import HTML
from IPython.core.display import Javascript
from IPython.core.display import Markdown
from olio.common import AD
from olio.widget import cleanupwidgets
from rich.console import Console


In [ ]:
# from bridget.routing import ar
from bridget.helpers import in_vscode_notebook


In [ ]:
# import psygnal.utils
# psygnal.utils.decompile()

# psygnal.utils.recompile()

----


In [ ]:
#| exporti

_n = '\n'

In [ ]:
cprint = (console := Console(width=120)).print

In [ ]:
if not in_vscode_notebook():
    NBStateFeedback.show(feedback=False, hide=True, debug=False)

----

# Replacing ipywidgets with Fasthtml

ipywidgets handles:
- **Layout**: ipywidgets uses python, JS -> replace with FastHTML, HTMX.
- **Comunication** kernel-frontend: ipywidgets uses Comm -> Bridget/HTTP-Ajax
- **Serialization**: ipywidgets uses Traitlets -> replace with pydantic, dataclasses, msgspec, pysignal.
- **Observer** pattern: ipywidgets uses Traitlets -> replace with custom or psignal.


In [ ]:
bridge_cfg.update(auto_show=True, auto_mount=True)
app, brt, rt = get_app(hooks=True, show_logger=True)

In [ ]:
brt.bridge.plugins

In [ ]:
dv = Div(cls='bridget slider')
dv.set(cls=f"{dv.get('class')} value")
with bridge_cfg(auto_show=False):
    display(dv)

In [ ]:
nb = brt.bridge.state
nb.cells

# Helpers

In [ ]:
def show_routes(app):
    cprint(app.routes)
    while app.routes and (app := getattr(app.routes[-1], 'app', None)):
        if hasattr(app, 'routes') and isinstance(app.routes, list): cprint(app.routes)
        else: break

## Link
> Replicate traitlets.link

In [ ]:
def link(pair1, pair2, transform=None):
    """
    Bidirectionally link two fields, possibly on the same object, using optional transform (forward, reverse).
    pair1, pair2: tuples of (object, attr_name)
    transform: (func1→2, func2→1), or None.
    """
    a_obj, a_field = pair1
    b_obj, b_field = pair2
    tf_ab = transform[0] if transform and transform[0] else (lambda v: v)
    tf_ba = transform[1] if transform and transform[1] else (lambda v: v)

    def on_a(val):
        new = tf_ab(val)
        if getattr(b_obj, b_field) != new:
            with getattr(a_obj.events, a_field).blocked():
                setattr(b_obj, b_field, new)

    def on_b(val):
        new = tf_ba(val)
        if getattr(a_obj, a_field) != new:
            with getattr(b_obj.events, b_field).blocked():
                setattr(a_obj, a_field, new)

    getattr(a_obj.events, a_field).connect(on_a)
    getattr(b_obj.events, b_field).connect(on_b)

In [ ]:
@psygnal.evented
@dataclass
class Item:
    x: float = 0.0
    y: float = 0.0


item = Item(x=32.0, y=0.0)

# x is Fahrenheit, y is Celsius
f_to_c = lambda f: (f - 32) * 5/9
c_to_f = lambda c: c * 9/5 + 32

link((item, "x"), (item, "y"), (f_to_c, c_to_f))

item.x = 212.0
test_eq(item.y, 100)

item.y = 0.0
test_eq(item.x, 32)

## _debug_req

In [ ]:
def _debug_req(name, url, method, body):
    return {
        'headers': {
            'HX-Request': 'true',
            'HX-Trigger-Name': name,
            'Content-Type': 'application/x-www-form-urlencoded'
        },
        'headerNames': {'hx-request': 'HX-Request', 'hx-current-url': 'HX-Current-URL', 
                        'hx-trigger-name': 'HX-Trigger-Name', 'hx-trigger-value': 'HX-Trigger-Value',
                        'content-type':'Content-Type'},
        'status': 0,
        'method': method,
        'url': url,
        'async': True,
        'timeout': 0,
        'withCredentials': False,
        'body': body,
    }

In [ ]:
@dataclass
class Person:
    name: str
    age: int = 0

    events: ClassVar[psygnal.SignalGroupDescriptor] = psygnal.SignalGroupDescriptor()

@dataclass
class Citizen(Person):
    country: str = 'USA'
    _city: str = 'New York'

@dataclass
class Employee(Citizen):
    company: str = 'Central Perk'

e = Employee(name='John', age=30)
psygnal.is_evented(Employee), psygnal.is_evented(e)

In [ ]:
e.events.signals

# BridgetRouteProvider

In [ ]:
def attrgetterd(*attrs, default=None):
    def _call(obj):
        oo = tuple(getattr(obj, attr, default) for attr in attrs)
        return oo[0] if len(oo) == 1 else oo
    return _call

In [ ]:
r = AD(name='a')
test_eq(attrgetterd('name')(r), 'a')
test_eq(attrgetterd('name', 'kind')(r), ('a', None))

In [ ]:
def types(o, stop=-1): return reversed(type(o).__mro__[:stop])

In [ ]:
[*types(APIRouterD(), None)]  # type: ignore

In [ ]:
if typing.TYPE_CHECKING:
    class Control: ...

def _cls_attr(cls: type, attr_name='_wrapper_class') -> str:
    if (n := cls.__name__).startswith('_'): return ''
    if hasattr(cls, attr_name): return getattr(cls, attr_name)
    if issubclass(cls, Control): return n.lower()
    return ''

_get_css = attrgetterd('_css')
def _cls_attrs(o, f=_cls_attr): return ' '.join(filter(None, (f(_) for _ in types(o)))).strip()

In [ ]:
class BridgetRouteProvider(RouteProvider):
    bridget = get_bridget()

    _updating_ = False
    @contextmanager
    def _updating(self): self._updating_ = True; yield; self._updating_ = False
    def setup_values(self): ...
    
    events: ClassVar[psygnal.SignalGroupDescriptor]
    def __init_subclass__(cls):
        # cls.bridget.clear_routes(cls)
        # cls.bridget.app.routes.clear()
        # cls.ar = ar = APIRouterD(f"/{cls.__name__}")
        # ar.__set_name__(cls, 'ar')
        if not 'events' in vars(cls): 
            cls.events = events = psygnal.SignalGroupDescriptor(warn_on_no_fields=False)
            events.__set_name__(cls, 'events')  # call explicitly: outside class definition
    def setup_events(self):
        if dataclasses.is_dataclass(self):
            fields = {fld.name:fld.metadata.get('bridget', {}) for fld in dataclasses.fields(self)}
        else: fields = {}
        for name in self.events.signals.keys():
            mtdt = fields.get(name, {})
            if mtdt.get('skip', False): continue
            mth = mtdt.get('on', None) or getattr(self, f"on_{name}", None)
            if not mth: mth = partial(getattr(self, 'on_attr'), attr=name)
            if isinstance(mth, (MethodType, partial)): self.events[name].connect(mth)
    def on_attr(self, nw: Any, old: Any, *, attr:str):
        if self.bridget and not self._updating_:
            with self._updating(): self.setup_values()
            self.bridget.bridge.commander.swap(f"#{self.wrapper_id()}", to_xml(self.__ft__()), swapStyle='innerHTML')

    ar = APIRouterD()
    _mounted = False
    def wrapper_id(self): return self.ar.name().replace(':', '_')
    def wrapper_class(self): return _cls_attrs(self)
    def style(self): return _cls_attrs(self, f=_get_css)
    def controls(self): return ()
    def wrapper(self): return Div(id=self.wrapper_id(), cls=self.wrapper_class(), style=self.style())
    def __ft__(self):
        if bridge_cfg.auto_mount and not self._mounted: get_bridget().mount(self, show=False)  # type: ignore
        return self.wrapper()(*self.controls())

    dh: DisplayHandle | None = None
    def _ipython_display_(self):
        # clear previous display; unlike ipywidgets, bridget shows only one view, MVC not welcomed here
        if self.dh is not None: self.dh.update(HTML())
        self.dh = self.bridget(self, display_id=self.dh)
        self.cell = this()

# Control

In [ ]:
class _Control(BridgetRouteProvider):
    disabled:bool=False; description:str=''
    def on_disabled(self, nw: Any, old: Any): ...
    def on_description(self, nw: Any, old: Any): ...


@dataclass
class Control(_Control):
    _: KW_ONLY
    disabled:bool = False
    description:str = ''
    
    def __post_init__(self):
        with self._updating(): self.setup_values()
        self.setup_events()

In [ ]:
# app.routes.clear()

In [ ]:
c = Control(disabled=True)
with bridge_cfg(auto_show=False):
    display(c.__ft__())

In [ ]:
show_routes(app)

# Value

In [ ]:
@dataclass
class Value(Control):
    value: Any = None

    @ar('/value', methods=['POST'], name='value')
    def changed(self, value:str):
        with self._updating(): self.value = value
        return self.value

    def value_control(self):
        return H.Input(name='value', value=self.value,
                hx_post=self.ar.value.to(), hx_trigger='input changed', hx_swap='none')
    def controls(self):
        ctl = self.value_control()
        if hasattr(ctl, '__ft__'): ctl = ctl.__ft__()
        if isinstance(ctl, (FT, Control)): return (ctl,)
        if isinstance(ctl, tuple): return ctl
        if isinstance(ctl, Iterable): return tuple(ctl)
        return (ctl,)

In [ ]:
app.routes.clear()

In [ ]:
v = Value('Hello, world!')
with bridge_cfg(auto_show=False):
    display(v.__ft__())
v

In [ ]:
show_routes(app)

In [ ]:
v.value

In [ ]:
v.value = 'Bye!'

# String

## Label

In [ ]:
# W.Label('Hello, World!')

```html
<div class="lm-Widget jupyter-widgets widget-inline-hbox widget-label">Hello, World!</div>
```

In [ ]:
@dataclass
class Label(Control):
    value: str = ''
    def controls(self): return (self.value, )

In [ ]:
v = Label('Hello, world!')
with bridge_cfg(auto_show=False):
    display(v.__ft__())
v

In [ ]:
v.value

## Text

In [ ]:
# W.Text('Hello, World!')

```html
<div class="lm-Widget jupyter-widgets widget-inline-hbox widget-text">
    <label class="widget-label" for="8ffde047-0ce3-4324-97a8-8f9bb6bb5a22" style="display: none;"></label>
    <input type="text" id="8ffde047-0ce3-4324-97a8-8f9bb6bb5a22" class="widget-input" placeholder="​">
</div>
```

In [ ]:
@dataclass
class Text(Value):
    value: str = ''
    @ar('/value', methods=['POST'], name='value')
    def changed(self, value:str):
        with self._updating(): self.value = value
    def value_control(self):
        return H.Input(name='value', value=self.value,
                hx_post=self.ar.value, hx_trigger='input changed', hx_swap='none')

In [ ]:
v = Text('Hello, world!')
with bridge_cfg(auto_show=False):
    display(v.__ft__())
v

In [ ]:
v.value

In [ ]:
v.value = 'Good-Bye to All That'

In [ ]:
show_routes(app)

## TextArea

In [ ]:
# W.Textarea('Hello, World!')

```html
<div class="lm-Widget jupyter-widgets widget-inline-hbox widget-textarea">
    <label class="widget-label" for="ea87d53a-15a1-474f-a14d-2f84302a10d5" style="display: none;"></label>
    <textarea rows="" id="ea87d53a-15a1-474f-a14d-2f84302a10d5" class="widget-input" placeholder="​"></textarea>
</div>
```

In [ ]:
@dataclass
class TextArea(Value):
    value: str = ''
    @ar('/value', methods=['POST'], name='value')
    def changed(self, value:str):
        with self._updating(): self.value = value
    def value_control(self):
        return H.Textarea(name='value', 
                hx_post=self.ar.value, hx_trigger='keyup changed throttle:100ms', hx_swap='none')(self.value)

In [ ]:
app.routes.clear()

In [ ]:
v = TextArea('Hello, world!')
with bridge_cfg(auto_show=False):
    display(v.__ft__())
v

In [ ]:
v.value

In [ ]:
v.value = 'Adieu!'

In [ ]:
show_routes(app)

## Combobox

In [ ]:
# W.Combobox(
#     value='Paul',
#     placeholder='Choose Someone',
#     options=['Paul', 'John', 'George', 'Ringo'],
#     description='Combobox:',
#     ensure_option=True,
#     disabled=False
# )

```html
<div class="lm-Widget jupyter-widgets widget-inline-hbox widget-text">
    <label class="widget-label" for="81e63e64-d351-4dd4-860a-6982c4d65bee" style="">Combobox:</label>
    <input type="text" id="81e63e64-d351-4dd4-860a-6982c4d65bee" class="widget-input" placeholder="Choose Someone" list="62fd0f57-8eef-4b16-97d8-2648d373e452">
    <datalist id="62fd0f57-8eef-4b16-97d8-2648d373e452">
        <option value="Paul"></option>
        <option value="John"></option>
        <option value="George"></option>
        <option value="Ringo"></option>
    </datalist>
</div>
```

In [ ]:
@dataclass
class Combobox(Value):
    value: str = ''
    options: list[str] = field(default_factory=list)
    def value_control(self):
        return (
            H.Input(type='text', name='value', list='abc-456', placeholder='Choose Someone', 
                hx_post=self.ar.value, hx_swap='none',
            ),
            H.Datalist(id='abc-456')(*map(lambda x:H.Option(value=x), self.options)),
        )

In [ ]:
v = Combobox('Ringo', ['Paul', 'John', 'George', 'Ringo'])
with bridge_cfg(auto_show=False):
    display(v.__ft__())
v

In [ ]:
v.value

In [ ]:
v.value = 'George'

In [ ]:
show_routes(app)

# Button

In [ ]:
# W.Button(
#     description='Click me',
#     disabled=False,
#     button_style='', # 'success', 'info', 'warning', 'danger' or ''
#     tooltip='Click me',
#     icon='check' # (FontAwesome names without the `fa-` prefix)
# )

```html
<button class="lm-Widget jupyter-widgets jupyter-button widget-button" title="Click me"><i class="fa fa-check"></i>Click me</button>
```

In [ ]:
@dataclass
class Button(Control):
    icon: str = ''
    def controls(self):
        return (H.Button(title=self.description)(H.I(cls=f"fa fa-{self.icon}"), self.description), )

In [ ]:
v = Button('check', description='Click me')
with bridge_cfg(auto_show=False):
    display(v.__ft__())
v

# Containers

### Box

In [ ]:
primes = list(map(int, """
2	3	5	7	11	13	17	19	23	29	31	37	41	43	47	53	59	61	67	71
73	79	83	89	97	101	103	107	109	113	127	131	137	139	149	151	157	163	167	173
179	181	191	193	197	199	211	223	227	229	233	239	241	251	257	263	269	271	277	281
283	293	307	311	313	317	331	337	347	349	353	359	367	373	379	383	389	397	401	409
419	421	431	433	439	443	449	457	461	463	467	479	487	491	499	503	509	521	523	541
547	557	563	569	571	577	587	593	599	601	607	613	617	619	631	641	643	647	653	659
661	673	677	683	691	701	709	719	727	733	739	743	751	757	761	769	773	787	797	809
811	821	823	827	829	839	853	857	859	863	877	881	883	887	907	911	919	929	937	941
947	953	967	971	977	983	991	997""".split()))
len(primes)

In [ ]:
# title_widget = W.HTML('<em>Box Example</em>')
# slider = W.IntSlider()
# W.Box([title_widget, slider])

In [ ]:
# W.HBox([title_widget, slider]) # flex-direction: row;

In [ ]:
# W.VBox([title_widget, slider]) # flex-direction: column;

In [ ]:
@dataclass
class Box(Control):
    children: list[Bridgeable] = field(default_factory=list)
    _css = 'display: flex; margin: 0;'

    def controls(self): return self.children
    def on_children(self, nw: Any, old: Any):
        dh = self.bridget(self, display_id=self.dh)
        if self.dh is None: self.dh = dh

@dataclass
class HBox(Box):
    _css = 'flex-direction: row; gap: 10px; column-gap: 10px;'

@dataclass
class VBox(Box):
    _css = 'flex-direction: column; row-gap: 5px;'

In [ ]:
v = Box(['Hello', Text('world!')])
with bridge_cfg(auto_show=False):
    display(v.__ft__())
v

In [ ]:
w = HBox(['Hello', Text('world!')])
with bridge_cfg(auto_show=False):
    display(w.__ft__())
w

In [ ]:
w = VBox(['Hello', Text('world!')])
with bridge_cfg(auto_show=False):
    display(w.__ft__())
w

In [ ]:
w = HBox()
with bridge_cfg(auto_show=False):
    display(w.__ft__())
w

In [ ]:
w.cell#nb.cell_by_did(w.dh.display_id)

In [ ]:
test_eq(w.children, [])
w.children = [vt1 := H.Text('asdf')]

test_eq(w.children, [vt1])
# test_eq(w.dhdl.contents(), f'<div id="{w.ar.name()}" class="widget">\n<text>asdf</text></div>\n')  # type: ignore
test_is('asdf' in w.cell.outputs[1]['data']['text/html'], True)

In [ ]:
w.children = [*w.children, vb1 :=H.Button('asdf')]

test_eq(w.children, [vt1, vb1])
test_is('button' in w.cell.outputs[1]['data']['text/html'], True)

In [ ]:
w.children = [*w.children, vb2 := Button('check', description='Click me')]

test_eq(w.children, [vt1, vb1, vb2])
test_is(vb2.ar.name() in w.cell.outputs[1]['data']['text/html'], True)

In [ ]:
# w = Widget(contents=[Button('asdf'), Text('asdf')])

In [ ]:
# help(Widget)

In [ ]:
show_routes(app)

In [ ]:
cprint(app.routes[1].app.routes)  # type: ignore

# Numeric

## IntText


In [ ]:
@dataclass
class IntText(Value):
    value: int = 0
    @ar('/value', methods=['POST'], name='value')
    def changed(self, value:int):
        with self._updating(): self.value = value
        # return the update to allow feedback
        return self.value or str(self.value)  # fasthtml inconditionally handles 0s from route handlers as falsy, not bona fide values
    def value_control(self):
        return H.Input(name='value', value=str(self.value), type='number', 
                # hx_post=f"{self.ar.to('changed')}", hx_trigger='input changed throttle:50ms', hx_swap='none')
                hx_post=self.ar.value, hx_trigger='input changed', hx_swap='none')

In [ ]:
app.routes.clear()

In [ ]:
v = IntText(5)
with bridge_cfg(auto_show=False):
    display(v.__ft__())
v

In [ ]:
v.value

In [ ]:
v.value = 10

In [ ]:
show_routes(app)

## BoundedInt

In [ ]:
@dataclass
class BoundedInt(IntText):
    min: int = 0
    max: int = 100
    step: int = 1

    def setup_values(self):
        if (val := self.value) < self.min: self.value = self.min
        elif val > self.max: self.value = self.max

    def value_control(self):
        return super().value_control()(min=str(self.min), max=str(self.max), step=str(self.step))

In [ ]:
v = BoundedInt(10, 5, 15)
with bridge_cfg(auto_show=False):
    display(v.__ft__())
v

In [ ]:
v.value

In [ ]:
v.max = 10
test_eq(v.value, 10)

## IntSlider

In [ ]:
@dataclass
class IntSlider(BoundedInt):
    readout: bool = True
    readout_format: str = 'd'
    
    _wrapper_class = 'slider'
    def value_control(self):
        return super().value_control()(type='range', hx_target='next text', hx_swap='textContent')
    def controls(self):
        return (H.Label(_for='value')('Scale'), _n,
                self.value_control(),
                H.Text(id='spanscale', style='inline')(self.value), _n)

In [ ]:
# brt.bridge.logger.show(clear=True)

In [ ]:
iv = IntSlider()
with bridge_cfg(auto_show=False):
    display(iv.__ft__())
iv

In [ ]:
# response = brt(_debug_req('value', iv.ar.value.to(), 'POST', 'value=50'))

In [ ]:
iv.value

In [ ]:
iv.value = 89

In [ ]:
iv.min = 20
test_eq(iv.value, 89)

In [ ]:
iv.max = 80
test_eq(iv.value, 80)

In [ ]:
iv2 = IntSlider(33, step=2)
iv2

In [ ]:
iv3 = IntSlider()
iv4 = IntSlider()
link((iv3, 'value'), (iv4, 'value'))

box = Div(style='display: flex; gap: 1em;')(iv3, iv4)
box

In [ ]:
iv3.value = 22
test_eq((iv3.value, iv4.value), (22, 22))

In [ ]:
iv5 = IntSlider(57)
iv6 = IntSlider()
link((iv5, 'value'), (iv6, 'value'))

box = HBox([iv5, iv6])
box

# Bool

In [ ]:
class _Bool(Value):
    value: bool = False
    @ar('/value', methods=['POST'], name='value')
    def changed(self, value:str='', abc:bool=False):
        with self._updating(): self.value = not self.value
        return ''

```HTML
<div class="lm-Widget jupyter-widgets widget-inline-hbox widget-checkbox">
    <label class="widget-label" style="">​</label>
    <label class="widget-label-basic">
        <input type="checkbox">
        <span title=""></span>
    </label>
</div>
```

In [ ]:
# w = W.Checkbox(True)
# w

In [ ]:
class Checkbox(_Bool):
    def value_control(self):
        return H.Div(cls='inline-hbox')(
            H.Label('Check')(
                Input(type='checkbox', name='value', checked=self.value, 
                    hx_post=self.ar.value, hx_trigger='click', 
                    hx_vals="js:{'abc':event.target.checked}", hx_swap='none')
            ),
        )

In [ ]:
# brt.bridge.logger.show(clear=True)

In [ ]:
# app.routes.clear()

In [ ]:
v = Checkbox(False)
with bridge_cfg(auto_show=False):
    display(v.__ft__())
v

In [ ]:
# response = brt(_debug_req('value', f"{v.ar.to('changed')}", 'POST', 'value=on&abc=true'))

In [ ]:
test_is(v.value, False)

In [ ]:
v.value = True

In [ ]:
test_is(v.value, True)

```HTML
<div class="lm-Widget p-Widget jupyter-widgets widget-valid widget-inline-hbox mod-invalid"><label class="widget-label" title="null" style="display: none;"></label><i class="fa fa-fw fa-times"></i><span class="widget-valid-readout widget-readout">Invalid</span></div>

<div class="lm-Widget jupyter-widgets widget-valid widget-inline-hbox mod-valid"><label class="widget-label" title="null" style="display: none;"></label><i class="fa fa-fw fa-check"></i><span class="widget-valid-readout widget-readout">Invalid</span></div>
```

In [ ]:
# w = W.Valid(True)
# w

In [ ]:
class Valid(_Bool):
    def value_control(self):
        return H.Div(cls='inline-hbox mod-invalid')(
            H.I(cls=f"fa fa-fw fa-{'check' if self.value else 'times'}"),
            H.Text(cls='readout')('valid' if self.value else 'invalid')
        )

In [ ]:
v = Valid(True)
with bridge_cfg(auto_show=False):
    display(v.__ft__())
v

In [ ]:
v.value

In [ ]:
v.value = False

In [ ]:
# w = W.ToggleButton(True, description='Click me')
# w

```html
<button class="lm-Widget jupyter-widgets jupyter-button widget-toggle-button mod-active" tabbable="null" title="null">Click me</button>
```

```html
<button class="lm-Widget jupyter-widgets jupyter-button widget-toggle-button" tabbable="null" title="null">Click me</button>
```

In [ ]:
class ToggleButton(_Bool):
    @ar('/value', methods=['POST'], name='value')
    def changed(self, value:bool):
        with self._updating(): self.value = not value
        return self.value_control()
    def value_control(self):
        return H.Button('Click me')(
            cls=f"button", name='value', 
            value=f"{'on' if self.value else 'off'}", style=f"{'background-color:darkgray' if self.value else ''}",
            hx_post=self.ar.value, hx_swap='outerHTML')

In [ ]:
# brt.bridge.logger.show(clear=True)

In [ ]:
# app.routes.clear()

In [ ]:
v = ToggleButton(False)
with bridge_cfg(auto_show=False):
    display(v.__ft__())
v

In [ ]:
# response = brt(_debug_req('value', f"{v.ar.to('changed')}", 'POST', 'value=on'))
# print(v.value)

# Selection

## Dropdown

In [ ]:
# W.Dropdown(
#     options=['1', '2', '3'],
#     value='2',
#     description='Number:',
#     disabled=False,
# )

```html
<div class="lm-Widget jupyter-widgets widget-inline-hbox widget-dropdown">
    <label class="widget-label" for="3f6a6063-c1fa-4f3d-966b-185eb49eb67f" style="">Number:</label>
    <select id="3f6a6063-c1fa-4f3d-966b-185eb49eb67f">
        <option data-value="1" value="1">1</option>
        <option data-value="2" value="2">2</option>
        <option data-value="3" value="3">3</option>
    </select>
</div>
```

In [ ]:
# W.Dropdown(
#     options=[('One', 1), ('Two', 2), ('Three', 3)],
#     value=2,
#     description='Number:',
# )

In [ ]:
@dataclass
class _Select(Value):
    options: list[tuple[str, Any]|Any] = field(default_factory=list)

    def __post_init__(self):
        self.options = [t if isinstance(t, tuple) else (str(t), t) for t in self.options]
        super().__post_init__()

    @cached_property
    def values(self) -> list[Any]: return [v for _,v in self.options]
    @property
    def index(self) -> int|None: return self.values.index(self.value) if self.value in self.values else None
    @index.setter
    def index(self, i:int): self.value = self.values[i] if i>=0 and i<len(self.options) else None
    
    def on_options(self, nw: Any):
        try: del self.values
        except Exception: pass
        if self.value not in self.values: self.value = None

    @ar('/value', methods=['POST'], name='value')
    def changed(self, value:str, index:int):
        with self._updating():
            # self.index = index
            self.value = self.options[index][1]
            self.setup_values()

In [ ]:
@dataclass
class Dropdown(_Select):
    def value_control(self):
        return H.Div(cls='inline-hbox')(
            H.Label(self.description)(
                H.Select(name='value', 
                    hx_post=self.ar.value, 
                    hx_vals="js:{'index':event.target.selectedIndex}", hx_swap='none')(
                    *[Option(value=opt[1], selected=i==self.index)(opt[0]) for i,opt in enumerate(self.options)]
                )
            )
        )

In [ ]:
# brt.bridge.logger.show(clear=True)

In [ ]:
# app.routes.clear()

In [ ]:
v = Dropdown(
    value=2,
    options=[('One', 1), ('Two', 2), ('Three', 3)]
)
with bridge_cfg(auto_show=False):
    display(v.__ft__())
v

In [ ]:
v.value, v.index

In [ ]:
v.value = 1

## Select

In [ ]:
# W.Select(
#     options=['Linux', 'Windows', 'macOS'],
#     value='macOS',
#     # rows=10,
#     description='OS:',
#     disabled=False
# )

```html
<div class="lm-Widget jupyter-widgets widget-inline-hbox widget-select">
    <label class="widget-label" for="a3c9d534-7c03-4948-9cf2-87dc8f353e1f" style="">OS:</label>
    <select id="a3c9d534-7c03-4948-9cf2-87dc8f353e1f" size="5">
        <option data-value="Linux" value="Linux">Linux</option>
        <option data-value="Windows" value="Windows">Windows</option>
        <option data-value="macOS" value="macOS">macOS</option>
    </select>
</div>
```

In [ ]:
@dataclass
class SelectArea(_Select):
    rows: int = 5

    def value_control(self):
        return H.Div(cls='inline-hbox')(
            H.Label(self.description)(
                H.Select(name='value', size=self.rows,
                    hx_post=self.ar.value, 
                    hx_vals="js:{'index':event.target.selectedIndex}", hx_swap='none')(
                    *[H.Option(value=opt[1], selected=i==self.index)(opt[0]) for i,opt in enumerate(self.options)]
                )
            )
        )

In [ ]:
# brt.bridge.logger.show(clear=True)

In [ ]:
# app.routes.clear()

In [ ]:
v = SelectArea(
    options=['Linux', 'Windows', 'macOS'],
    value='macOS',
    # rows=10,
    description='OS:',
)
with bridge_cfg(auto_show=False):
    display(v.__ft__())
v

In [ ]:
v.value, v.index

In [ ]:
v.value = 'Windows'

In [ ]:
# response = brt(_debug_req('value', f"{v.ar.to('changed')}", 'POST', 'value=Windows&index=1'))
# print(v.value)

## SelectMultiple

In [ ]:
# W.SelectMultiple(
#     options=['Apples', 'Oranges', 'Pears'],
#     value=['Oranges'],
#     #rows=10,
#     description='Fruits',
#     disabled=False
# )

```html
<div class="lm-Widget jupyter-widgets widget-inline-hbox widget-select widget-select-multiple">
    <label class="widget-label" for="c3af7603-7856-431f-ad6a-15ede273e09f" style="">Fruits</label>
    <select multiple="" id="c3af7603-7856-431f-ad6a-15ede273e09f" size="5">
        <option data-value="Apples" value="Apples">Apples</option>
        <option data-value="Oranges" value="Oranges">Oranges</option>
        <option data-value="Pears" value="Pears">Pears</option>
    </select>
</div>
```

In [ ]:
@dataclass
class SelectMultiple(_Select):
    value: list[Any] = field(default_factory=list)
    # index: list[int] = field(default_factory=list)
    rows: int = 5

    @property
    def index(self) -> list[int]: return [self.values.index(_) for _ in self.value]
    @index.setter
    def index(self, ii:list[int]): self.value = [self.values[_] for _ in ii if _ in range(len(self.options))]

    @ar('/value', methods=['POST'], name='value')
    def changed(self, value:list[str]=[], index:list[int]=[]):
        with self._updating():
            # self.index = index
            self.value = [self.values[_] for _ in index if _ in range(len(self.options))]
            self.setup_values()

    def value_control(self):
        return H.Div(cls='inline-hbox')(
            H.Label(self.description)(
                H.Select(name='value', size=self.rows, multiple=True, 
                    hx_post=self.ar.value, 
                    hx_vals="js:{'index':[...event.target.selectedOptions].map(i=>i.index)}", hx_swap='none')(
                    *[H.Option(value=opt[1], selected=i in self.index)(opt[0]) for i,opt in enumerate(self.options)]
                )
            )
        )

In [ ]:
# brt.bridge.logger.show(clear=True)

In [ ]:
# app.routes.clear()

In [ ]:
v = SelectMultiple(
    options=['Apples', 'Oranges', 'Pears'],
    value=['Pears'],
    description='Fruits',
)
with bridge_cfg(auto_show=False):
    display(v.__ft__())
v

In [ ]:
v.value, v.index

## RadioButtons

In [ ]:
# W.RadioButtons(
#     options=['pepperoni', 'pineapple', 'anchovies'],
# #    value='pineapple', # Defaults to 'pineapple'
# #    layout={'width': 'max-content'}, # If the items' names are long
#     description='Pizza topping:',
#     disabled=False
# )

```html
<div class="lm-Widget widget-radio">
    <label class="widget-label" title="null" style="">Pizza topping:</label>
    <div class="widget-radio-box widget-radio-box-vertical" style="margin-bottom: 2px;">
        <label>pepperoni<input type="radio" value="0" data-value="pepperoni"></label>
        <label>pineapple<input type="radio" value="1" data-value="pineapple"></label>
        <label>anchovies<input type="radio" value="2" data-value="anchovies"></label>
    </div>
</div>
```

In [ ]:
app.routes.clear()

In [ ]:
@dataclass
class RadioButtons(_Select):
    @ar('/value', methods=['POST'], name='value')
    def changed(self, index:int):
        with self._updating():
            self.value = self.options[index-1][1]
    def value_control(self):
        n = self.ar.name().split(':')[-1]
        wid = self.wrapper_id()
        return H.Div(cls='inline-hbox')(
            H.Label(self.description)(
                Div(name='value', hx_post=self.ar.value, 
                    hx_vals="js:{'index':[...document.querySelectorAll('#" + wid + " input')].filter(e=>e.checked)[0].value}",
                    hx_trigger=f"click from:(#{wid} input)", hx_swap='none')(
                    *[H.Label(opt[0], 
                        H.Input(type='radio', value=i+1, name=n, checked=i==self.index)) 
                    for i,opt in enumerate(self.options)]
                )
            )
        )

In [ ]:
v = RadioButtons(
    options=['Larry', 'Moe', 'Curly'],
    value='Moe',
    description='3S:',
)
with bridge_cfg(auto_show=False):
    display(v.__ft__())
v

In [ ]:
# brt.bridge.logger.show(clear=True)

In [ ]:
v.value, v.index

In [ ]:
show_routes(app)

In [ ]:
# W.Box(
#     [
#         W.Label(value='Pizza topping with a very long label:'), 
#         W.RadioButtons(
#             options=[
#                 'pepperoni', 
#                 'pineapple', 
#                 'anchovies', 
#                 'and the long name that will fit fine and the long name that will fit fine and the long name that will fit fine '
#             ],
#             layout={'width': 'max-content'}
#         )
#     ]
# )

```html
<div class="widget-box">
    <div class="lm-Widget jupyter-widgets widget-inline-hbox widget-label">Pizza topping with a very long label:</div>
    <div class="radio" style="width: max-content;">
        <label class="widget-label" title="null" style="display: none;"></label>
        <div class="radio-box radio-box-vertical">
            <label>pepperoni<input type="radio" value="0" data-value="pepperoni"></label>
            <label>pineapple<input type="radio" value="1" data-value="pineapple"></label>
            <label>anchovies<input type="radio" value="2" data-value="anchovies"></label>
            <label>and the long name that will fit fine and the long name that will fit fine and the long name that will fit fine <input type="radio" value="3" data-value="and%20the%20long%20name%20that%20will%20fit%20fine%20and%20the%20long%20name%20that%20will%20fit%20fine%20and%20the%20long%20name%20that%20will%20fit%20fine%20"></label>
        </div>
    </div>
</div>
```

## ToggleButtons

In [ ]:
# W.ToggleButtons(
#     options=['Slow', 'Regular', 'Fast'],
#     description='Speed:',
#     disabled=False,
#     button_style='', # 'success', 'info', 'warning', 'danger' or ''
#     tooltips=['Description of slow', 'Description of regular', 'Description of fast'],
# #     icons=['check'] * 3
# )


```html
<div class="lm-Widget widget-toggle-buttons">
    <label class="widget-label" title="null" style="">Speed:</label>
    <div>
        <button type="button" class="widget-toggle-button jupyter-button mod-active" data-value="Slow" value="0" title="Description of slow">Slow<i></i></button>
        <button type="button" class="widget-toggle-button jupyter-button" data-value="Regular" value="1" title="Description of regular">Regular<i></i></button>
        <button type="button" class="widget-toggle-button jupyter-button" data-value="Fast" value="2" title="Description of fast">Fast<i></i></button>
    </div>
</div>
```

# Simple widgets (Traitlets)


## Value

In [ ]:
# class Value(T.HasTraits, Control):
#     value = T.Any()
    
#     @T.observe('value')
#     def on_value(self, _):
#         if self.bridget and not self._updating_:
#             self.bridget.bridge.commander.swap(f"#{self.ar.name()}", to_xml(self.__ft__()), swapStyle='innerHTML')

In [ ]:
# app.routes.clear()

In [ ]:
# v = Value(value='Hello, world!')
# with bridge_cfg(auto_show=False):
#     display(v.__ft__())
# v

In [ ]:
# v.value

In [ ]:
# v.value = 'bye!'

## Numeric

In [ ]:
# class Numeric(Value):
#     value: int
#     min, max, step = None, None, None
#     input_type = 'number'
#     hx_target, hx_swap = None, 'none'
#     @ar('/value', methods=['POST'])
#     def changed(self, value:int):
#         with self._updating(): self.value = value
#         return self.value  # fasthtml handles 0s from route handlers as falsy, not bona fide values
#     def __ft__(self):
#         return Input(type=self.input_type, name='value', 
#                     value=self.value, min=self.min, max=self.max, step=self.step,
#                     hx_post=f"/{self.ar.to()}/value", hx_trigger='input changed throttle:100ms', 
#                     hx_target=self.hx_target, hx_swap=self.hx_swap)

In [ ]:
# class _Int(_Value):
#     value: int
#     min, max, step = None, None, None
#     input_type = 'number'
#     hx_target, hx_swap = None, 'none'
#     @ar('/value', methods=['POST'])
#     def changed(self, value:int):
#         with self._updating(): self.value = value
#         return self.value  # fasthtml handles 0s from route handlers as falsy, not bona fide values
#     def value_control(self):
#         return Input(type=self.input_type, name='value', value=self.value, min=self.min, max=self.max, step=self.step,
#                 hx_post=f"/{self.ar.to()}/value", hx_trigger='input changed throttle:100ms', 
#                 hx_target=self.hx_target, hx_swap=self.hx_swap)

### IntText

In [ ]:
# class IntText(Value, _Int):
#     value = T.CInt()

In [ ]:
# v = IntText(value=5)
# with bridge_cfg(auto_show=False):
#     display(v.__ft__())
# v

In [ ]:
# v.value

In [ ]:
# v.value = 17

### BoundedInt

In [ ]:
# class BoundedInt(IntText):
#     min = T.CInt(0)
#     max = T.CInt(100)
#     step = T.Int(1)

#     def value_control(self): return super().value_control()(min=self.min, max=self.max, step=self.step)

In [ ]:
# v = BoundedInt(value=10, min=5, max=15)
# with bridge_cfg(auto_show=False):
#     display(v.__ft__())
# v

In [ ]:
# v.value

In [ ]:
# v.max = 17

### IntSlider

In [ ]:
# class IntSlider(BoundedInt):
#     readout:bool=True; readout_format:str='d'
    
#     _wrapper_class = 'slider'
#     def value_control(self):
#         return Input(type='range', name='value', min=self.min, max=self.max, step=self.step, value=self.value,
#                 hx_post=f"/{self.ar.to()}/value", hx_trigger='input changed throttle:100ms', 
#                 hx_target='next text', hx_swap='textContent')
#     def setup_controls(self):
#         return (Label(_for='value')('Scale'), _n,
#                 self.value_control(),
#                 Text(id='spanscale', style='inline')(self.value), _n)

In [ ]:
# iv = IntSlider()
# with bridge_cfg(auto_show=False):
#     display(iv.__ft__())
# iv

In [ ]:
# iv.value

In [ ]:
# iv.value = 77

In [ ]:
# iv2 = IntSlider(step=2)
# iv2

In [ ]:
# iv3 = IntSlider()
# iv4 = IntSlider()
# # brt.mount(iv3, show=False)
# # brt.mount(iv4, show=False)
# T.link((iv3, 'value'), (iv4, 'value'))

# box = Div(style='display: flex; gap: 1em;')(iv3, iv4)
# box

In [ ]:
# iv3.value = 22
# test_eq(iv4.value, 22)

# Simple Widget (psignal) - Not working

# Output

In [ ]:
out = W.Output()
out

In [ ]:
msg_id = None
with out:
    msg_id = out.msg_id
    out.clear_output()
    display(Text('Hello, world!'))
msg_id

In [ ]:
out.append_display_data(HTML('<div id="asdf">asdf</div>'))
out.append_display_data(Div(style='color: red;')(Text('Hi!')))
out.append_display_data(Markdown('Hi!'))

In [ ]:
# o = HTML('<div id="asdf">asdf</div>')
# display(o)

In [ ]:
from IPython.utils.capture import RichOutput, CapturedIO, capture_output

In [ ]:
with capture_output() as io:
    display(Div(style='color: red;')(Text('Hi!')), HTML('<div id="asdf">asdf</div>'), 'Hi!', Markdown('Hi!'))
# io.outputs
# io.show()

In [ ]:
io()

In [ ]:
def _capture(*objs, **kwargs):
    from IPython.core.interactiveshell import InteractiveShell
    if not InteractiveShell.initialized():
        raise RuntimeError('IPython not initialized')
    outs = []
    format = InteractiveShell.instance().display_formatter.format
    for obj in objs:
        format_dict, md_dict = format(obj)
        if not format_dict:
            # nothing to display (e.g. _ipython_display_ took over)
            outs.append((None, None))
            continue
        outs.append((format_dict, md_dict))
    return outs

_capture(Div(style='color: red;')(Text('Hi!')), HTML('<div id="asdf">asdf</div>'), 'Hi!', Markdown('Hi!'))

In [ ]:
def _capture_output(*objs, **kwargs):
    with capture_output() as io:
        display(*objs, **kwargs)
    return io

capt = _capture_output(Div(style='color: red;')(Text('Hi!')), HTML('<div id="asdf">asdf</div>'), 'Hi!', Markdown('Hi!'))
capt()
for c in capt.outputs:
    print(c._repr_mimebundle_())

In [ ]:
class Output(Widget):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.contents = []
        Bridget().mount(self, show=False)
    # def __ft__(self):
    #     return super().__ft__()(id=self.idx)
    # def _ipython_display_(self):
    #     Bridget()(self, display_id=self.idx)
    def clear_output(self, wait=False):
        self.contents = []
        if not wait and self.dhdl: self.dhdl.update()
    # def capture(self, clear_output=False, *clear_args, **clear_kwargs):
    #     pass
    # @contextmanager
    # def managed_resource(*args, **kwds):
    #     # Code to acquire resource, e.g.:
    #     resource = acquire_resource(*args, **kwds)
    #     try:
    #         yield resource
    #     finally:
    #         # Code to release resource, e.g.:
    #         release_resource(resource)

    def __enter__(self):
        self._capt = capture_output()
        self._cio = self._capt.__enter__()
    def __exit__(self, etype, evalue, tb):
        bridge_cfg.current_did = None
        self._capt.__exit__(etype, evalue, tb)
        outputs, self._cio, self._capt = self._cio.outputs, None, None
        if not self.dhdl:
            dhdl = display(display_id=True)
            self.dhdl = DisplayId(dhdl.display_id)
        dhdl = self.dhdl.display_id
        for c in outputs:
            mime = {'data': c.data, 'metadata': c.metadata}
            self.contents.append(mime)
            display(mime['data'], metadata=mime['metadata'], raw=True, update=True, display_id=dhdl)
    
    # def _flush(self):
    #     pass
    def append_stdout(self, text):
        brt = self.bridget
        brt.swap(f"#{self.ar.name}", text+'<br>', swapStyle='beforeend')
        self.contents.append(text)
    def append_stderr(self, text):
        brt = self.bridget
        brt.swap(f"#{self.ar.name}", text+'<br>', swapStyle='beforeend')
        self.contents.append(text)
    def append_display_data(self, display_object):
        with capture_output() as io:
            display(display_object)
        mimebundle = io.outputs[0]._repr_mimebundle_()
        html = mimebundle[0]['text/html']
        brt = self.bridget
        brt.swap(f"#{self.ar.name}", html, swapStyle='beforeend')
        self.contents.append(display_object)

In [ ]:
out = Output()
with bridge_cfg(auto_show=False):
    display(out.__ft__())
out

In [ ]:
out(Text('Hello, world!'))

In [ ]:
out.append_stdout('Hello, universe!')


In [ ]:
out.append_display_data(Div(style='color: red; padding: 0px;')(Text('Hello, red!')))
out.append_display_data(Button('adfrt'))

In [ ]:
out.contents


In [ ]:
out = Output()
with out:
    display('adfrt')
# out.contents


In [ ]:
out.append_display_data(Div(style='color: red; padding: 0px;')(Text('Hello, red!')))

In [ ]:
out.contents

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
with out:
    fig = plt.figure(figsize=(4, 2.67))
    plt.plot([1,2,3,4])
    # plt.show()
out.contents

In [ ]:
# with out:
#     plt.plot([1,2,3,4])
#     plt.show()

----

In [ ]:
out = Output(layout={'border': '1px solid black'})
out


In [ ]:
with out:
    for i in range(10):
        print(i, 'Hello world!')

In [ ]:
from IPython.display import YouTubeVideo
with out:
    display(YouTubeVideo('eWzY2nGfkXk'))

In [ ]:
with out:
    display(widgets.IntSlider())

In [ ]:
out = widgets.Output(layout={'border': '1px solid black'})
out.append_stdout('Output appended with append_stdout')
out.append_display_data(YouTubeVideo('eWzY2nGfkXk'))
out

In [ ]:
@out.capture()
def function_with_captured_output():
    print('This goes into the output widget')
    raise Exception('As does this')
    
function_with_captured_output()

In [ ]:
a = widgets.IntSlider(description='a')
b = widgets.IntSlider(description='b')
c = widgets.IntSlider(description='c')
def f(a, b, c):
    print('{}*{}*{}={}'.format(a, b, c, a*b*c))

out = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c})

widgets.HBox([widgets.VBox([a, b, c]), out])

# interactive_output


# interactive

In [ ]:
def f(a, b):
    display(a + b)
    return a+b

In [ ]:
w = W.interactive(f, a=10, b=20)
w

In [ ]:
%matplotlib inline
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np

def f(m, b):
    plt.figure(2)
    x = np.linspace(-10, 10, num=1000)
    plt.plot(x, m * x + b)
    plt.ylim(-5, 5)
    plt.show()

interactive_plot = interactive(f, m=(-2.0, 2.0), b=(-3, 3, 0.5))
output = interactive_plot.out#children[-1]
output.layout.height = '420px'
interactive_plot

# interact

In [ ]:
def f(x):
    return x

In [ ]:
interact(f, x=10);

In [ ]:
interact(f, x=True);

In [ ]:
interact(f, x='Hi there!');

In [ ]:
@interact(x=True, y=1.0)
def g(x, y):
    return (x, y)


In [ ]:
IntSlider(min=-10, max=30, step=1, value=10)

In [ ]:
interact(f, x=widgets.IntSlider(min=-10, max=30, step=1, value=10));


# interact_manual

# Colophon
----


In [ ]:
import fastcore.all as FC
import nbdev
from nbdev.clean import nbdev_clean


In [ ]:
if FC.IN_NOTEBOOK:
    nb_path = '50_widget.ipynb'
    nbdev_clean(nb_path)
    nbdev.nbdev_export(nb_path)
